<div align="center">
  <img src="../assets/images/hackathon.png" alt="Holistic AI Hackathon Logo" width="600"/>
</div>

**Event**: [hackathon.holisticai.com](https://hackathon.holisticai.com)

---


# Tutorial 2: Building Custom Tools

**Create powerful custom tools for your agents**

## What You'll Learn

1. **What are tools** - Understanding the tool interface
2. **Simple tools** - Using the `@tool` decorator
3. **Multiple tools** - Multi-step reasoning with tools
4. **See agent reasoning** - Watch tool calls in action

## Why Custom Tools?

Tools extend your agent's capabilities:
- **Search** - Web search, database queries
- **Data** - APIs, calculations, file operations
- **Generation** - Images, code, documents
- **Safety** - Input validation, content filtering

---

## Prerequisites

- Basic Python knowledge
- Recommended: Completed `01_basic_agent.ipynb`
- Time: ~20 minutes

**Note:** This tutorial is completely self-contained and uses only official packages!

## Step 0: Install Dependencies

Run this cell to install all required packages.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load from .env file in parent directory
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables")

# Verify API keys
print("\n🔑 API Key Status:")
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("  ✅ Holistic AI Bedrock credentials loaded")
elif os.getenv('OPENAI_API_KEY'):
    print("  ⚠️  OpenAI API key loaded")
else:
    print("  ⚠️  No API keys found")

print("\n📁 Working directory:", Path.cwd())

# Import Holistic AI Bedrock helper
import sys
try:
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import get_chat_model
    print("\n✅ Holistic AI Bedrock helper loaded")
except ImportError:
    print("\n⚠️  Could not import from core - will use OpenAI only")

# Import official packages
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_core.tools import BaseTool

print("\n✅ All imports successful!")


## Step 1: Setup Environment

**Recommended:**
Set up Holistic AI Bedrock Proxy API credentials in `.env`:

```bash
HOLISTIC_AI_TEAM_ID=tutorials_api
HOLISTIC_AI_API_TOKEN=your-token-here
```

[API Guide](../assets/api-guide.pdf)

**Alternative (Optional):**
If you prefer OpenAI:

```bash
OPENAI_API_KEY=your-openai-api-key-here
```

**Note:** The tutorial uses Holistic AI Bedrock by default (recommended). OpenAI is optional - to use it, call get_chat_model(model_name, use_openai=True) and set OPENAI_API_KEY.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# ============================================
# OPTION 1: Set API keys directly (Quick Start)
# ============================================
# Uncomment and set your keys here:
# Recommended: Holistic AI Bedrock
# os.environ["HOLISTIC_AI_TEAM_ID"] = "tutorials_api"
# os.environ["HOLISTIC_AI_API_TOKEN"] = "your-token-here"
# Alternative: OpenAI (optional)
# os.environ["OPENAI_API_KEY"] = "your-openai-key-here"

# ============================================
# OPTION 2: Load from .env file (Recommended)
# ============================================
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    print("📄 Loaded configuration from .env file")
else:
    print("⚠️  No .env file found - using environment variables or hardcoded keys")

# ============================================
# Import Holistic AI Bedrock helper function
# ============================================
# Import from core module (recommended)
import sys
try:
    sys.path.insert(0, '../core')
    from react_agent.holistic_ai_bedrock import HolisticAIBedrockChat, get_chat_model
    print("✅ Holistic AI Bedrock helper function loaded")
except ImportError:
    print("⚠️  Could not import from core - will use OpenAI only")

# Import official packages
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Verify API keys
print("\n🔑 API Key Status:")
if os.getenv('HOLISTIC_AI_TEAM_ID') and os.getenv('HOLISTIC_AI_API_TOKEN'):
    print("  ✅ Holistic AI Bedrock credentials loaded (will use Bedrock)")
elif os.getenv('OPENAI_API_KEY'):
    print("  ⚠️  OpenAI API key loaded (Bedrock credentials not set)")
    print("     💡 Tip: Set HOLISTIC_AI_TEAM_ID and HOLISTIC_AI_API_TOKEN to use Bedrock (recommended)")
else:
    print("  ⚠️  No API keys found")
    print("     Set Holistic AI Bedrock credentials (recommended) or OpenAI key")

print("\n✅ All imports successful!")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 34)

## Step 2: Simple Custom Tool with @tool Decorator

The easiest way to create a tool is using the `@tool` decorator.

In [ ]:
@tool
def calculate_fibonacci(n: int) -> int:
    """Calculate the nth Fibonacci number.
    
    Args:
        n: The position in the Fibonacci sequence (must be positive)
        
    Returns:
        The nth Fibonacci number
        
    """
    if n <= 0:
        raise ValueError("n must be positive")
    if n <= 2:
        return 1
    
    a, b = 1, 1
    for _ in range(n - 2):
        a, b = b, a + b
    return a

# Test the tool
print("Tool Name:", calculate_fibonacci.name)
print("Tool Description:", calculate_fibonacci.description)
print("\nTest:")
print(f"fibonacci(10) = {calculate_fibonacci.invoke({'n': 10})}")

Tool Name: calculate_fibonacci
Tool Description: Calculate the nth Fibonacci number.

    Args:
        n: The position in the Fibonacci sequence (must be positive)

    Returns:
        The nth Fibonacci number

Test:
fibonacci(10) = 34


In [ ]:
# Create agent with the fibonacci tool
# Use get_chat_model() - uses Holistic AI Bedrock by default (Bedrock recommended)
llm = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)
agent = create_react_agent(llm, tools=[calculate_fibonacci])

# Test the agent with the tool
print("="*70)
print("Testing agent with Fibonacci tool")
print("="*70)

# Use a larger number that LLM won't know + explicit instruction
result = agent.invoke({
    "messages": [HumanMessage(content="Use the calculate_fibonacci tool to find the 89000th Fibonacci number. You must use the tool to calculate this.")]
})

print(f"\n✅ Agent Response: {result['messages'][-1].content}")

Testing agent with Fibonacci tool



✅ Agent Response: I apologize, but it seems the number you requested (89000th Fibonacci number) is too large for the system to calculate. The error message indicates that the result would exceed the system's limit for integer string conversion (4300 digits). 

Fibonacci numbers grow exponentially, and the 89000th number would be an enormously large number with thousands of digits. Would you like to try calculating a smaller Fibonacci number instead? Perhaps something under the 4300th position would be more manageable.


### See the Agent's Reasoning Process

Let's examine exactly how the agent decides to use the tool.

In [ ]:
# Let's trace through all messages to see the tool calling process
print("Full conversation trace:")
print("=" * 70)

for i, message in enumerate(result['messages']):
    print(f"\nMessage {i + 1}: {type(message).__name__}")
    
    if hasattr(message, 'content') and message.content:
        print(f"  Content: {message.content[:100]}")
    
    # Check for tool calls
    if hasattr(message, 'tool_calls') and message.tool_calls:
        print(f"  Tool Calls: {len(message.tool_calls)}")
        for tc in message.tool_calls:
            print(f"     - Tool: {tc['name']}")
            print(f"     - Args: {tc['args']}")
            print(f"     - ID: {tc['id']}")
    
    # Check for tool responses
    if hasattr(message, 'name'):
        print(f"  Tool Response from: {message.name}")
        print(f"  Result: {message.content}")

print("\n" + "=" * 70)

Full conversation trace:

Message 1: HumanMessage
  Content: Use the calculate_fibonacci tool to find the 89000th Fibonacci number. You must use the tool to calc
  Tool Response from: None
  Result: Use the calculate_fibonacci tool to find the 89000th Fibonacci number. You must use the tool to calculate this.

Message 2: AIMessage
  Tool Calls: 1
     - Tool: calculate_fibonacci
     - Args: {'n': 89000}
     - ID: toolu_bdrk_013NBU8NKayW93LFJsuJZJqn
  Tool Response from: None
  Result: 

Message 3: ToolMessage
  Content: Error: ValueError('Exceeds the limit (4300 digits) for integer string conversion; use sys.set_int_ma
  Tool Response from: calculate_fibonacci
  Result: Error: ValueError('Exceeds the limit (4300 digits) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit')
 Please fix your mistakes.

Message 4: AIMessage
  Content: I apologize, but it seems the number you requested (89000th Fibonacci number) is too large for the s
  Tool Response fro

### Understanding the Flow

The agent follows this process:

1. **User asks a question** - "Use the calculate_fibonacci tool to find the 47th Fibonacci number"
2. **Agent analyzes** - Recognizes it needs to use the calculate_fibonacci tool
3. **Agent calls tool** - `calculate_fibonacci(n=47)`
4. **Tool executes** - Returns `2971215073`
5. **Agent responds** - Uses the result to answer the user

This is the **ReAct pattern**: Reason → Act → Observe → Respond

**Note:** We use larger numbers (like 47) and explicit instructions to ensure the agent calls tools instead of using memorized values.

## Deep Dive: How Tool Descriptions Work

### The Agent's Decision Process

When you give an agent tools, the LLM doesn't "know" what the tools do. It relies entirely on:

1. **Tool Name** - A descriptive function name
2. **Docstring** - The description and parameter details
3. **Parameter Types** - Type hints help the LLM understand expected inputs

Here's what happens behind the scenes:

```
User Question → LLM analyzes → Reads ALL tool descriptions → Decides which tool(s) to call
```

The LLM receives tool descriptions in this format:

```json
{
  "name": "calculate_fibonacci",
  "description": "Calculate the nth Fibonacci number.\n\nArgs:\n  n: The position in the Fibonacci sequence",
  "parameters": {
    "type": "object",
    "properties": {
      "n": {"type": "integer"}
    }
  }
}
```

**Key Insight:** Good descriptions = Better tool selection!

### Writing Effective Tool Descriptions

Follow these best practices:

In [ ]:
# Example: Good vs Bad Tool Descriptions

# ❌ BAD: Vague description
@tool
def process_data(x: int) -> int:
    """Process data."""  # LLM doesn't know what this does!
    return x * 2

# ✅ GOOD: Clear, specific description
@tool
def double_number(x: int) -> int:
    """Multiply a number by 2.
    
    Args:
        x: The number to double
        
    Returns:
        The input number multiplied by 2
        
    Example:
        double_number(5) returns 10
    """
    return x * 2

# ❌ BAD: Missing context
@tool
def search(q: str) -> str:
    """Search."""
    return f"Results for {q}"

# ✅ GOOD: Explains what, where, and when to use
@tool  
def search_company_database(query: str) -> str:
    """Search the internal company database for employee records.
    
    Use this tool when you need to find information about employees,
    departments, or organizational structure.
    
    Args:
        query: Search terms (employee name, department, role, etc.)
        
    Returns:
        Matching records from the company database
    """
    return f"Database results for {query}"

print("✅ Tool description examples created!")
print("\nKey Takeaways:")
print("  1. Be specific about what the tool does")
print("  2. Explain when to use it (if not obvious)")
print("  3. Document parameters clearly")
print("  4. Include examples when helpful")

✅ Tool description examples created!

Key Takeaways:
  1. Be specific about what the tool does
  2. Explain when to use it (if not obvious)
  3. Document parameters clearly
  4. Include examples when helpful


### Best Practices Checklist

When writing tool descriptions:

✅ **DO:**
- Use clear, descriptive function names
- Write detailed docstrings explaining what, when, and why
- Include parameter descriptions with expected formats
- Add examples for complex tools
- Specify units (e.g., "temperature in Celsius")
- Mention limitations or constraints

❌ **DON'T:**
- Use vague names like `process()`, `handle()`, `do_stuff()`
- Leave out docstrings or write minimal ones
- Assume the LLM "knows" what your tool does
- Use jargon without explanation

**Remember:** The LLM treats your tool as a black box. Your description is the ONLY way it knows what the tool does!

## Step 3: Multiple Tools with Multi-Step Reasoning

Now that you understand how tool descriptions work, let's see how agents use multiple tools together.

In [ ]:
@tool
def add_numbers(a: float, b: float) -> float:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
        
    Returns:
        Sum of a and b
    """
    result = a + b
    print(f"  Computing: {a} + {b} = {result}")
    return result

@tool
def multiply_numbers(a: float, b: float) -> float:
    """Multiply two numbers together.
    
    Args:
        a: First number
        b: Second number
        
    Returns:
        Product of a and b
    """
    result = a * b
    print(f"  Computing: {a} × {b} = {result}")
    return result

print("Math tools created!")
print("  - add_numbers")
print("  - multiply_numbers")

Math tools created!
  - add_numbers
  - multiply_numbers


### Multi-Step Reasoning Example

In [ ]:
# Create agent with multiple tools
# Use get_chat_model() - uses Holistic AI Bedrock by default (Bedrock recommended)
llm = get_chat_model("claude-3-5-sonnet")  # Uses Holistic AI Bedrock (recommended)
math_agent = create_react_agent(
    llm,
    tools=[calculate_fibonacci, add_numbers, multiply_numbers]
)

# Ask a complex question requiring multiple steps
print("Question: Calculate (fibonacci(35) + fibonacci(36)) × 2")
print("\nWatch the agent work through this step by step...\n")

result = math_agent.invoke({
    "messages": [HumanMessage(content="Use the tools to calculate: (fibonacci(35) + fibonacci(36)) multiplied by 2. You must use calculate_fibonacci, add_numbers, and multiply_numbers tools.")]
})

print("\n" + "=" * 70)
print("Final Answer:")
print(result['messages'][-1].content)
print("=" * 70)

Question: Calculate (fibonacci(35) + fibonacci(36)) × 2

Watch the agent work through this step by step...

  Computing: 5702887.0 + 9227465.0 = 14930352.0
  Computing: 14930352.0 × 2.0 = 29860704.0

Final Answer:
I'll break down the calculation:
1. First fibonacci(35) = 5702887
2. Then fibonacci(36) = 9227465
3. Adding these numbers: 5702887 + 9227465 = 14930352
4. Finally multiplying by 2: 14930352 × 2 = 29860704

The final result is 29860704.


### Inspect the Complete Tool Call Chain

In [ ]:
print("\nDetailed Tool Call Analysis:")
print("=" * 70)

tool_call_count = 0
for i, msg in enumerate(result['messages']):
    print(f"\nStep {i + 1}: {type(msg).__name__}")
    
    # User message
    if hasattr(msg, 'type') and msg.type == 'human':
        print(f"  User asks: {msg.content[:80]}...")
    
    # AI message with tool calls
    elif hasattr(msg, 'tool_calls') and msg.tool_calls:
        tool_call_count += len(msg.tool_calls)
        print(f"  AI decides to call {len(msg.tool_calls)} tool(s):")
        for j, tc in enumerate(msg.tool_calls, 1):
            print(f"     {j}. {tc['name']}({tc['args']})")
    
    # Tool response
    elif hasattr(msg, 'name'):
        print(f"  Tool '{msg.name}' returns: {msg.content}")
    
    # AI final response
    elif hasattr(msg, 'content') and msg.content and not hasattr(msg, 'tool_calls'):
        print(f"  AI responds: {msg.content[:80]}...")

print(f"\nTotal tool calls made: {tool_call_count}")
print("=" * 70)


Detailed Tool Call Analysis:

Step 1: HumanMessage
  User asks: Use the tools to calculate: (fibonacci(35) + fibonacci(36)) multiplied by 2. You...

Step 2: AIMessage
  AI decides to call 1 tool(s):
     1. calculate_fibonacci({'n': 35})

Step 3: ToolMessage
  Tool 'calculate_fibonacci' returns: 5702887

Step 4: AIMessage
  AI decides to call 1 tool(s):
     1. calculate_fibonacci({'n': 36})

Step 5: ToolMessage
  Tool 'calculate_fibonacci' returns: 9227465

Step 6: AIMessage
  AI decides to call 1 tool(s):
     1. add_numbers({'a': 5702887, 'b': 9227465})

Step 7: ToolMessage
  Tool 'add_numbers' returns: 14930352.0

Step 8: AIMessage
  AI decides to call 1 tool(s):
     1. multiply_numbers({'a': 14930352.0, 'b': 2})

Step 9: ToolMessage
  Tool 'multiply_numbers' returns: 29860704.0

Step 10: AIMessage
  Tool 'None' returns: I'll break down the calculation:
1. First fibonacci(35) = 5702887
2. Then fibonacci(36) = 9227465
3. Adding these numbers: 5702887 + 9227465 = 14930352
4. Finall

## Summary

Congratulations! You've learned:

### Key Concepts

1. **@tool Decorator** - Easiest way to create custom tools
2. **Tool Descriptions** - Important for agent to understand when to use tools
3. **Multi-Step Reasoning** - Agents can chain multiple tool calls
4. **Message Tracing** - See exactly how agents reason and act

### What's Next?

Continue learning:
- **03_structured_output.ipynb** - Get JSON responses with Pydantic
- **04_model_monitoring.ipynb** - Track costs and performance

---

**Pro Tip**: Good tool descriptions are critical! The agent uses them to decide which tool to call.